### Versions
- Unfortunately, `version` is not stored in the metadata of each original netcdf file nor in the name of the netcdf file. It is not in the `Dataset` name. When downloading the netcdf files via Globus, for example, the original path information is lost.  
- Data handles, aka persistent identifiers, called `tracking_id`s in the CMIP6 CV, are stored in the metadata of each netcdf file
- The Cloud zarr stores, which are aggregations of netcdf files corresponding to a particular dataset, store the `tracking_id`s of the constituent files in metadata
- The `tracking_id`s have been registered with the hdl.handle.net web service through links to the [DKRZ web service](https://handle-esgf.dkrz.de/) and a Proxy Server [REST API](https://www.handle.net/proxy_servlet.html) which will return `version` when given a `tracking_id`
- This notebook both demonstrates the use of the REST API and documents various problems/issues with this process

Our Cloud CMIP6 Catalogs records a `version` for each dataset.  This `version` has been obtained by a query to the Handle REST API using the constituent `tracking_id`s

In [2]:
import pandas as pd
import zarr
import fsspec

In [3]:
from myidentify import gsurl2tracks, tracks2version, tracks2source, jdict2source, id2jdict, tracks2cloudversion
from myutilities import search_df
from mysearch import esgf_search
import myconfig

In [4]:
dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')

In [5]:
def gsurl2search(gsurl):
    values = gsurl[11:-1].split('/')
    keys = myconfig.target_keys
    return dict(zip(keys,values))

In [6]:
# standard example
#gsurl = 'gs://cmip6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Oyr/expc/gr/'
#gsurl = 'gs://cmip6/CMIP/E3SM-Project/E3SM-1-0/historical/r1i1p1f1/Amon/tas/gr/'
#gsurl = 'gs://cmip6/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Amon/tas/gn/'
gsurl = 'gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/1pctCO2/r1i1p1f1/Amon/hfls/gn/v20200225'
version_cat = dfcat[dfcat.zstore == gsurl].version.values[0]
print('current version from GC catalog = ',version_cat)

tracks = gsurl2tracks(gsurl)
(version,jdict) = tracks2version(tracks)
print('latest version from handler = ', version)

asearch = gsurl2search(gsurl)
dfs = esgf_search(asearch, toFilter = False)
version_ESGF = list(set(dfs.version_id))
print('version(s) available from ESGF = ', version_ESGF)

#source_urls =tracks2source(tracks) 
#source_urls

current version from GC catalog =  20200225


current version from GC tracks =  ['20200225']
latest version from handler =  20200225
version(s) available from ESGF =  ['v20200225']


In [13]:
#gsurl = 'gs://cmip6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/3hr/uas/gn/'  # just created, but scripts say there is a newer version!!!!

gsurl = 'gs://cmip6/ScenarioMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/ssp370/r1i1p1f1/3hr/uas/gn/' # duplicate dataset_ids and versions 
                                                                # since two files are in both datasets and one file is only in the most recent
tracks = gsurl2tracks(gsurl)
tracking_ids = tracks.split('\n')
tracking_ids

['hdl:21.14100/d5aae306-16fe-4a2f-9706-8c41b4d20584',
 'hdl:21.14100/43133a86-783d-4d67-a73f-c512d2e27582',
 'hdl:21.14100/4fd3a7ec-d1ab-421b-963e-bcf82fcd8cdb']

In [5]:
# Now, what if we try this for another? This example has two features. The tracking_ids of the netcdf files are not unique and this is a replacement version.
gsurl = 'gs://cmip6/ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/ts/gn/'
tracks = gsurl2tracks(gsurl)
(version,jdict) = tracks2version(tracks)
print('version = ', version)

# But since the tracking_ids were not unique, we won't get all of the urls!!
source_urls = tracks2source(tracks) 
source_urls

/home/naomi/BigRewrite/myidentify.py:26: UserWarning: 

netcdf file tracking_ids are NOT UNIQUE!
['hdl:21.14100/33cbdc29-fbc9-44ab-9e09-5dc7824441cf', 'hdl:21.14100/33cbdc29-fbc9-44ab-9e09-5dc7824441cf']

  warnings.warn("\n" + msg)


cloud version from tracks =  20200528
version =  20200528


['http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/ts/gn/v20200528/ts_Amon_CESM2_ssp370_r4i1p1f1_gn_206501-210012.nc']

In [14]:
# This example is a Dataset where there are three versions (and an ES-DOC ERRATA link from second version)
gsurl = 'gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r1i1p1f1/Amon/tasmin/gr/'

version_cat = dfcat[dfcat.zstore == gsurl].version.values[0]
print('current version from GC catalog = ',version_cat)

tracks = gsurl2tracks(gsurl)
(version,jdict) = tracks2version(tracks)
print('latest version from handler = ', version)

asearch = gsurl2search(gsurl)
dfs = esgf_search(asearch, toFilter = False)
version_ESGF = list(set(dfs.version_id))
print('version(s) available from ESGF = ', version_ESGF)

#source_urls =tracks2source(tracks) 
#source_urls

current version from GC catalog =  20190926
current version from GC tracks =  20190926


/home/naomi/BigRewrite/myidentify.py:61: UserWarning: 

*** Newer version exists, see: http://hdl.handle.net/hdl:21.14100/480d0915-c4de-3b4a-89da-dbce9ace46ce

  warnings.warn(f'\n\n*** Newer version exists, see: http://hdl.handle.net/{ds_id}\n')
/home/naomi/BigRewrite/myidentify.py:61: UserWarning: 

*** Newer version exists, see: http://hdl.handle.net/hdl:21.14100/b7fc3bc4-2489-3627-b8ce-bf665b908fb6

  warnings.warn(f'\n\n*** Newer version exists, see: http://hdl.handle.net/{ds_id}\n')


latest version from handler =  20200310
version(s) available from ESGF =  ['v20200310', 'v20190926']


In [ ]:
gsurl = 'gs://cmip6/CMIP/THU/CIESM/historical/r1i1p1f1/Amon/tasmin/gr/'

In [6]:
dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')

df = search_df(dfcat,table_id='Amon',experiment_id='historical',variable_id='tas')
df['member'] = [int(s.split('r')[-1].split('i')[0]) for s in df['member_id']]
df = df.sort_values(by=['member'])
df = df.reset_index(drop=True)

len(df)

/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1552

In [8]:
for index, row in df.iterrows():
    if index > 14:
        continue
    gsurl = row['zstore']
    version = row['version']
    print(gsurl)
    
    tracks = gsurl2tracks(gsurl)
    (version_new,jdict) = tracks2cloudversion(tracks)
    print('\t',version, version_new)

gs://cmip6/CMIP/AS-RCEC/TaiESM1/historical/r1i1p1f1/Amon/tas/gn/
	 20200623 ['20200623']
gs://cmip6/CMIP/UA/MCM-UA-1-0/historical/r1i1p1f1/Amon/tas/gn/
	 20190731 ['20190731']
gs://cmip6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/Amon/tasmin/gr1/
	 20190610 ['20190610']
gs://cmip6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/Amon/tasmax/gr1/
	 20190610 ['20190610']
gs://cmip6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/Amon/tas/gr1/
	 20190610 ['20190610']
gs://cmip6/CMIP/INM/INM-CM4-8/historical/r1i1p1f1/Amon/tasmin/gr1/
	 20190530 ['20190530']
gs://cmip6/CMIP/INM/INM-CM4-8/historical/r1i1p1f1/Amon/tasmax/gr1/
	 20190530 ['20190530']
gs://cmip6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/tas/gr/
	 20180803 ['20180803']
gs://cmip6/CMIP/INM/INM-CM4-8/historical/r1i1p1f1/Amon/tas/gr1/
	 20190530 ['20190530']
gs://cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/tasmax/gn/
	 20190627 ['20190627']
gs://cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/tas/gn/


In [5]:
# Here is a really bad one:

gsurl = 'gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r12i1p1f1/day/hurs/gr/'

# https://handle-esgf.dkrz.de/lp/21.14100/388fda69-61ed-44f0-af3d-73d20dab3502   'handle is not accessible'
#version_cat = dfcat[dfcat.zstore == gsurl].version.values[0]
#print('current version from GC catalog = ',version_cat)

tracks = gsurl2tracks(gsurl)
#print(tracks)
(version,jdict) = tracks2version(tracks)
print('latest version from handler = ', version)

asearch = gsurl2search(gsurl)
dfs = esgf_search(asearch, toFilter = False)
version_ESGF = list(set(dfs.version_id))
print('version(s) available from ESGF = ', version_ESGF)

#source_urls =tracks2source(tracks) 
#source_urls'

/home/naomi/BigRewrite/myidentify.py:61: UserWarning: multiple dataset_ids correspond to the dataset tracking_ids!
hdl:21.14100/e3c5b869-c4d2-3d40-9585-5dd1ba09a9f8;hdl:21.14100/0aecf69a-2b74-32f4-96ff-879669b90960
  warnings.warn(f'multiple dataset_ids correspond to the dataset tracking_ids!\n{ds_tracking_id}')
/home/naomi/BigRewrite/myidentify.py:69: UserWarning: error resolving tracking_id hdl:21.14100/0aecf69a-2b74-32f4-96ff-879669b90960
  warnings.warn(f'error resolving tracking_id {dsid}')


current version from GC tracks =  ['20200315']
latest version from handler =  20200315
version(s) available from ESGF =  ['v20200315']


In [13]:
 id2jdict('hdl:21.14100/b8eec705-a216-4868-911e-f4b7d188e11f')

{'URL': 'https://handle-esgf.dkrz.de/lp/21.14100/b8eec705-a216-4868-911e-f4b7d188e11f',
 'AGGREGATION_LEVEL': 'FILE',
 'FIXED_CONTENT': 'TRUE',
 'FILE_NAME': 'hurs_day_EC-Earth3_historical_r12i1p1f1_gr_20140101-20141231.nc',
 'FILE_SIZE': '161893254',
 'IS_PART_OF': 'hdl:21.14100/e3c5b869-c4d2-3d40-9585-5dd1ba09a9f8;hdl:21.14100/0aecf69a-2b74-32f4-96ff-879669b90960',
 'FILE_VERSION': '1',
 'CHECKSUM': '391dc8728d86929978d86e809af71fafeb704ebb87ad4c15a5634b0e10178a34',
 'CHECKSUM_METHOD': 'SHA256',
 'URL_ORIGINAL_DATA': '<locations><location href="http://esgf.bsc.es/thredds/fileServer/esg_dataroot/a1tk-CMIP-r12/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r12i1p1f1/day/hurs/gr/v20201230/hurs_day_EC-Earth3_historical_r12i1p1f1_gr_20140101-20141231.nc" publishedOn="2020-12-30T18:11:22.384+00:00" host="esgf.bsc.es" dataset="hdl:21.14100/0aecf69a-2b74-32f4-96ff-879669b90960" /></locations>',
 'URL_REPLICA': '<locations><location href="http://esgf-data1.llnl.gov/thredds/fileServer/htt

In [9]:
tracks.split('\n')[-1:]

['hdl:21.14100/b8eec705-a216-4868-911e-f4b7d188e11f']

In [8]:
import xarray as xr
xr.open_zarr(fsspec.get_mapper('gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/1pctCO2/r1i1p1f1/Amon/hfls/gn/v20200225'))

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 192, lon: 288, time: 1800)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 0001-01-16 12:00:00 ... 0150-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1800, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    hfls       (time, lat, lon) float32 dask.array<chunksize=(292, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 0701-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     182500.0
    cmor_version:              3.5.0
    contact:                   Dr. Wei-Liang Lee (leelupin@gate.sinica.edu.tw)
    creation_date:             2020-01-17T03:03:45Z
    data_specs_version:        01.00.31
    experiment:                1 percent per year increase in CO2
    experiment_id:             1pctCO2
    external_variables:        areacella
    forcing_index:             1
    frequency:                 mon
    further_info_url:          https://furtherinfo.es-doc.org/CMIP6.AS-RCEC.T...
    grid:                      finite-volume grid with 0.9x1.25 degree lat/lo...
    grid_label:                gn
    history:                   2020-01-17T03:03:45Z ; CMOR rewrote data to be...
    initialization_index:      1
    institution:               Research Center for Environmental Changes, Aca...
    institution_id:            AS-RCEC
    license:                   CMIP6 model data produced by NCC is licensed u...
    mip_era:                   CMIP6
    model_id:                  TaiESM1
    nominal_resolution:        100 km
    parent_activity_id:        CMIP
    parent_experiment_id:      piControl
    parent_mip_era:            CMIP6
    parent_source_id:          TaiESM1
    parent_sub_experiment_id:  none
    parent_time_units:         days since 1850-01-01
    parent_variant_label:      r1i1p1f1
    physics_index:             1
    product:                   model-output
    realization_index:         1
    realm:                     atmos
    references:                http://cclics.rcec.sinica.edu.tw/index.php/tai...
    run_variant:               N/A
    source:                    TaiESM 1.0 (2018): \naerosol: SNAP (same grid ...
    source_id:                 TaiESM1
    source_type:               AOGCM AER BGC
    status:                    2020-05-01;created; by gcs.cmip6.ldeo@gmail.com
    sub_experiment:            none
    sub_experiment_id:         none
    table_id:                  Amon
    table_info:                Creation Date:(24 July 2019) MD5:0bb394a356ef9...
    title:                     TaiESM1 output prepared for CMIP6
    tracking_id:               hdl:21.14100/813dbc9a-249f-4cde-a56c-fea0a42a5eb5
    variable_id:               hfls
    variant_label:             r1i1p1f1

In [10]:
import gcsfs
import xarray as xr

# create a MutableMapping from a store URL
fs = gcsfs.GCSFileSystem(token='anon', access='read_only')
mapper = fs.get_mapper("gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/1pctCO2/r1i1p1f1/Amon/hfls/gn/v20200225/")
# make sure to specify that metadata is consolidated
ds = xr.open_zarr(mapper, consolidated=True)


In [16]:
# get the path to a specific zarr store
zstore = df_subset.zstore.values[-1]
mapper = gcsfs.get_mapper(zstore)
# open using xarray
ds = xr.open_zarr(mapper, consolidated=True)

NameError: name 'df_subset' is not defined

In [17]:
df = pd.read_csv("https://cmip6.storage.googleapis.com/pangeo-cmip6.csv")
df_subset = df.query("activity_id=='CMIP' & table_id=='Amon' & variable_id=='tas'")
zstore = df_subset.zstore.values[-1]

/usr/local/python/anaconda3/envs/pangeo-Oct2019/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
zstore = df_subset.zstore.values[-1]
mapper = fs.get_mapper(zstore)
# open using xarray
ds = xr.open_zarr(mapper, consolidated=True)


In [19]:
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 80, longitude: 96, time: 6000)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat_bnds   (latitude, bnds) float64 dask.array<chunksize=(80, 2), meta=np.ndarray>
  * latitude   (latitude) float64 -88.29 -86.07 -83.84 ... 83.84 86.07 88.29
    lon_bnds   (longitude, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * longitude  (longitude) float64 0.0 3.75 7.5 11.25 ... 348.8 352.5 356.2
  * time       (time) object 0001-01-17 00:00:00 ... 0500-12-17 00:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(6000, 2), meta=np.ndarray>
Data variables:
    areacella  (latitude, longitude) float64 dask.array<chunksize=(80, 96), meta=np.ndarray>
    height     float64 ...
    tas        (time, latitude, longitude) float32 dask.array<chunksize=(600, 80, 96), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2 
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   1533000.0
    branch_time_in_parent:  0.0
    comment:                 Used lowest model level about 25m above surface ...
    contact:                GEOS-CMIP@email.arizona.edu           
    creation_date:          2019-05-04T00:00:00Z
    data_specs_version:     01.00.28
    experiment:             pre-industrial control                           ...
    experiment_id:          piControl
    external_variables:     areacella
    forcing_index:          1
    frequency:              mon
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.UA.MCM-UA-1-...
    grid:                   lat-lon
    grid_label:             gn
    history:                File was produced by a FORTRAN program
    initialization_index:   1
    institution:            Department of Geosciences, University of Arizona,...
    institution_id:         UA
    license:                CMIP6 model data produced by the U of Arizona is ...
    mip_era:                CMIP6
    nominal_resolution:     250 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl-spinup
    parent_mip_era:         CMIP6
    parent_source_id:       MCM-UA-1-0
    parent_time_units:      days since 0001-01-01 (noleap)
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             Delworth, Thomas L., Ronald J Stouffer, Keith W D...
    source:                 Manabe Climate Model at U of Arizona             ...
    source_id:              MCM-UA-1-0
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               Amon
    title:                  UArizona MCM-UA-1-0 piControl           
    tracking_id:            hdl:21.14100/d82343e0-612b-4045-bc05-4b7ea8d7a3b0...
    variable_id:            tas
    variant_info:           1850 equiv CO2 and aerosol forcing    
    variant_label:          r1i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu